In [1]:
import os
import ccxt
import schedule
import time
from tabulate import tabulate
from dotenv import load_dotenv
from datetime import datetime, timedelta
import ntplib
import numpy as np
import logging
import asyncio
import nest_asyncio
import threading
import csv
from ntplib import NTPClient  # Import NTPClient
import matplotlib.pyplot as plt
 
# Load environment variables from .env file with the correct encoding
load_dotenv
 
# Create an empty list to store trading signals
trading_signals = []

symbol_to_analyze = 'BTC/USDT:USDT'
leverage = 10  # Define your desired leverage
amount = 1  # Define your desired amount
volatility_multiplier = 1.0  # You can adjust the value as needed

TAKE_PROFIT_PERCENTAGE = 1.35
STOP_LOSS_PERCENTAGE = 1.35
 
# Fetch API credentials from environment variables
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
PASSPHRASE = os.getenv('PASSPHRASE')
 
# Initialize the KuCoin Futures exchange instance
exchange = ccxt.kucoinfutures({
    'apiKey': API_KEY,
    'secret': SECRET_KEY,
    'password': PASSPHRASE,
    'enableRateLimit': True  # Adjust as needed
})
      
# Create an NTP client instance
ntp_client = NTPClient()
 
# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Set your desired log level
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler("trading_log.txt"),  # Log to a file
        logging.StreamHandler()  # Log to the console
    ]
)
 
# Create a logger instance
logger = logging.getLogger(__name__)

def calculate_atr(high_prices, low_prices, close_prices, period=14):
    if high_prices is None or low_prices is None or close_prices is None:
        print("Error: One or more data sources are None.")
        return None

    # Check if any of the input lists are empty
    if not high_prices.all() or not low_prices.all() or not close_prices.all():
        print("Error: One or more data sources are empty.")
        return None

    # Calculate True Range (TR)
    tr = [max(hl, hc, lc) - min(hl, hc, lc) for hl, hc, lc in zip(high_prices, close_prices, low_prices)]
    
    # Calculate the Average True Range (ATR) using a period (e.g., 14)
    atr = np.mean(tr[-period:])
    return atr
 
def calculate_rsi(data, atr, period=14):
    # Calculate the differences between consecutive closing prices
    price_diff = np.diff(data)

    # Calculate the positive and negative price changes
    positive_changes = np.where(price_diff > 0, price_diff, 0)
    negative_changes = np.where(price_diff < 0, -price_diff, 0)

    # Smooth the average gains and losses using ATR
    smoothed_avg_gain = np.mean(positive_changes[-period:])
    smoothed_avg_loss = np.mean(negative_changes[-period:])

    # Avoid division by zero by checking if smoothed_avg_loss is not zero
    if smoothed_avg_loss != 0:
        rs = smoothed_avg_gain / smoothed_avg_loss
        rsi = 100 - (100 / (1 + rs))
    else:
        rsi = 100  # Set RSI to a high value when smoothed_avg_loss is zero

    return rsi

def calculate_obv(close_prices, volume_data):
    obv = [0]  # Initialize OBV with 0
    for i in range(1, len(close_prices)):
        if close_prices[i] > close_prices[i - 1]:
            obv.append(obv[-1] + volume_data[i])  # Adding volume on up days
        elif close_prices[i] < close_prices[i - 1]:
            obv.append(obv[-1] - volume_data[i])  # Subtracting volume on down days
        else:
            obv.append(obv[-1])  # No change in OBV on unchanged days
    return obv
 
def calculate_smoothed_imbalance(data, alpha=0.1):
    smoothed_data = [data[0]]  # Initialize with the first data value
    for i in range(1, len(data)):
        smoothed_data.append(alpha * data[i] + (1 - alpha) * smoothed_data[i - 1])
    return smoothed_data

def calculate_order_book_liquidity_dynamic(symbol, volatility_multiplier):
    try:
        # Fetch order book data for the specified symbol
        order_book = exchange.fetch_order_book(symbol)

        # Extract bid and ask data
        bids = order_book['bids']
        asks = order_book['asks']

        # Calculate market volatility (you can replace this with your preferred indicator)
        market_volatility = calculate_market_volatility(order_book)

        # Determine the dynamic percentage range based on market volatility
        dynamic_range_percentage = 2.0 * market_volatility  # Adjust the multiplier as needed

        # Calculate the price range around the market price
        market_price = (float(bids[0][0]) + float(asks[0][0])) / 2.0
        lower_bound = market_price * (1 - dynamic_range_percentage / 100)
        upper_bound = market_price * (1 + dynamic_range_percentage / 100)

        # print(f"Market Price: {market_price}")
        # print(f"Dynamic Range Percentage: {dynamic_range_percentage}%")
        # print(f"Lower Bound: {lower_bound}")
        # print(f"Upper Bound: {upper_bound}")

        # Filter order book data within the dynamic range
        filtered_bids = [(float(price), float(quantity)) for price, quantity in bids if lower_bound <= float(price) <= upper_bound]
        filtered_asks = [(float(price), float(quantity)) for price, quantity in asks if lower_bound <= float(price) <= upper_bound]

        # Calculate cumulative order volume at different price percentiles
        total_bid_volume = sum(quantity for price, quantity in filtered_bids)
        total_ask_volume = sum(quantity for price, quantity in filtered_asks)

        # print(f"Total Bid Volume within Dynamic Range: {total_bid_volume}")
        # print(f"Total Ask Volume within Dynamic Range: {total_ask_volume}")

        return market_price, dynamic_range_percentage, lower_bound, upper_bound, total_bid_volume, total_ask_volume

    except Exception as e:
        print(f"Error calculating order book liquidity: {e}")
        return None, None, None, None, None, None

import numpy as np

def calculate_market_volatility(order_book, period=14):
    try:
        # Extract high and low prices from the order book
        high_prices = [float(order[0]) for order in order_book['bids']]
        low_prices = [float(order[0]) for order in order_book['asks']]

        # Check if there's enough data to calculate ATR
        if len(high_prices) < period or len(low_prices) < period:
            raise ValueError("Not enough data to calculate ATR")

        # Calculate True Range (TR)
        tr = [max(hl, hc, lc) - min(hl, hc, lc) for hl, hc, lc in zip(high_prices, low_prices, low_prices[1:])]

        # Calculate the Average True Range (ATR) using a period (e.g., 14)
        atr = np.mean(tr[-period:])
        
        # Print informative message about ATR calculation
        print(f"Market Volatility Calculation:")
        print(f" - Using {period} periods")
        print(f" - Latest High Prices: {high_prices[-period:]}")
        print(f" - Latest Low Prices: {low_prices[-period:]}")
        print(f" - True Ranges: {tr[-period:]}")
        print(f" - Average True Range (ATR): {atr:.4f}")

        return atr

    except Exception as e:
        print(f"Error calculating market volatility: {e}")
        return None


def fetch_ohlcv_and_analyze_order_book(symbol, depth=100, max_retries=3):
    retries = 0
    # Initialize a list to store historical imbalance percentages
    historical_imbalance_percentage = []
    historical_obv = []  # Initialize a list to store historical OBV values

    rsi = None
    current_imbalance_percentage = None
    close_prices = None
    high_prices = None
    low_prices = None
    bids = None
    asks = None

    while retries < max_retries:
        try:
            # Use NTP to synchronize your system's time
            response = ntp_client.request('time.google.com')
            current_time = datetime.fromtimestamp(response.tx_time)

            # Fetch OHLCV data for ATR and TR calculation
            ohlcv_data = exchange.fetch_ohlcv(symbol, '4h')  # Adjust timeframe as needed
            close_prices = np.array([item[4] for item in ohlcv_data])
            high_prices = np.array([item[2] for item in ohlcv_data])
            low_prices = np.array([item[3] for item in ohlcv_data])

            # Fetch volume data
            volume_data = np.array([item[5] for item in ohlcv_data])

            # Calculate OBV
            obv = calculate_obv(close_prices, volume_data)

            # Calculate True Range (TR)
            tr = [max(hl, hc, lc) - min(hl, hc, lc) for hl, hc, lc in zip(high_prices, close_prices, low_prices)]

            # Calculate Average True Range (ATR) using a period (e.g., 14)
            atr = np.mean(tr[-14:])

            # Calculate RSI using close prices
            rsi = calculate_rsi(close_prices, atr)

            # Fetch the order book for the specified symbol and depth
            order_book = exchange.fetch_order_book(symbol, limit=20)
            bids = order_book['bids']
            asks = order_book['asks']

            # Extract bid prices and quantities
            bid_prices = [bid[0] for bid in bids]
            bid_quantities = [bid[1] for bid in bids]

            # Extract ask prices and quantities
            ask_prices = [ask[0] for ask in asks]
            ask_quantities = [ask[1] for ask in asks]

            # Create subplots for bids and asks
            plt.figure(figsize=(10, 4))

            # Plot bids in green
            plt.subplot(1, 2, 1)
            plt.barh(range(len(bid_prices)), bid_quantities, tick_label=bid_prices, color='green')
            plt.title('Top 20 Bids')
            plt.xlabel('Quantity')
            plt.ylabel('Price')

            # Plot asks in red
            plt.subplot(1, 2, 2)
            plt.barh(range(len(ask_prices)), ask_quantities, tick_label=ask_prices, color='red')
            plt.title('Top 20 Asks')
            plt.xlabel('Quantity')
            plt.ylabel('Price')

            # Show the plots
            plt.tight_layout()
            plt.show()

            # Calculate the total volume of bids and asks
            total_bids_volume = sum(bid[1] for bid in bids)
            total_asks_volume = sum(ask[1] for ask in asks)

            # Calculate the current order book imbalance percentage
            current_imbalance_percentage = ((total_bids_volume - total_asks_volume) / (total_bids_volume + total_asks_volume)) * 100

            # Calculate the weighted order book imbalance using the midpoint of bid and ask prices
            weighted_bids_volume = sum(bid[1] * (bid[0] + ask[0]) / 2 for bid, ask in zip(bids, asks))
            weighted_asks_volume = sum(ask[1] * (bid[0] + ask[0]) / 2 for bid, ask in zip(bids, asks))

            if weighted_bids_volume + weighted_asks_volume != 0:
                current_weighted_imbalance_percentage = (weighted_bids_volume - weighted_asks_volume) / (weighted_bids_volume + weighted_asks_volume) * 100
            else:
                current_weighted_imbalance_percentage = 0  # Default value when the total volume is zero

            # Calculate OBV trend direction and strength metrics
            obv_value = obv[-1]
            obv_direction = "Neutral"
            obv_strength = "Weak"
            if obv_value > obv[-2]:
                obv_direction = "Bullish (OBV Rising)"
                if obv_value - obv[-2] > 0:
                    obv_strength = "Strong"
            elif obv_value < obv[-2]:
                obv_direction = "Bearish (OBV Falling)"
                if obv[-2] - obv_value > 0:
                    obv_strength = "Strong"

            # Calculate OBV divergence with price
            obv_divergence = "No Divergence"
            if obv[-1] > obv[-2] and close_prices[-1] < close_prices[-2]:
                obv_divergence = "Bullish Divergence (OBV Leading): OBV is rising while prices are falling."
                divergence_strength = "Weak"
                if obv[-1] - obv[-2] > 0.05 * close_prices[-2]:
                    divergence_strength = "Strong"
                obv_divergence += f" Strength: {divergence_strength}"
            elif obv[-1] < obv[-2] and close_prices[-1] > close_prices[-2]:
                obv_divergence = "Bearish Divergence (OBV Leading): OBV is falling while prices are rising."
                divergence_strength = "Weak"
                if obv[-2] - obv[-1] > 0.05 * close_prices[-2]:
                    divergence_strength = "Strong"
                obv_divergence += f" Strength: {divergence_strength}"

            # Print order book analysis results along with enhanced OBV analysis
            print(
                f"Order Book Analysis for {symbol} - Imbalance: {current_imbalance_percentage:.2f}% "
                f"(Weighted Imbalance: {current_weighted_imbalance_percentage:.2f}%) - RSI: {rsi:.2f} - "
                f"OBV Direction: {obv_direction} ({obv_strength}) - "
                f"OBV Divergence: {obv_divergence}"
            )
            
            # Append the current imbalance percentage to the historical list
            historical_imbalance_percentage.append(current_imbalance_percentage)
            historical_obv.append(obv[-1])

            # Calculate smoothed order book imbalance using EMA
            smoothed_imbalance = calculate_smoothed_imbalance(historical_imbalance_percentage)

            # Generate trading signal and proposed entry price based on RSI and order book imbalance
            trading_signal, proposed_entry_price, take_profit_price, stop_loss_price = generate_trading_signal(
                rsi,
                current_imbalance_percentage,
                close_prices,
                high_prices,
                low_prices,
                bids,
                asks
            )

            print("Trading Signal:", trading_signal)
            if proposed_entry_price:
                print("Proposed Entry Price:", proposed_entry_price)
                print("Take Profit Price:", take_profit_price)
                print("Stop Loss Price:", stop_loss_price)
            print("=" * 50)

            # Exit the retry loop if data is successfully fetched and analyzed
            break

        except Exception as e:
            retries += 1
            print(f"Error fetching or analyzing order book: {e}")
            print(f"Retrying... ({retries}/{max_retries})")
            time.sleep(10)  # Wait for 10 seconds before retrying

    # Return the calculated values
    return rsi, current_imbalance_percentage, close_prices, high_prices, low_prices, bids, asks

def calculate_take_profit_and_stop_loss(entry_price, leverage, take_profit_percentage, stop_loss_percentage):
    # Calculate the leverage-adjusted entry price
    leverage_adjusted_entry_price = entry_price / leverage

    # Calculate take profit and stop loss prices based on the leverage-adjusted entry price
    take_profit_price = round(entry_price * (1 + TAKE_PROFIT_PERCENTAGE / 100), 8)
    stop_loss_price = round(entry_price * (1 - STOP_LOSS_PERCENTAGE / 100), 8)

    return take_profit_price, stop_loss_price
 
def generate_trading_signal(rsi, imbalance_percentage, close_prices, high_prices, low_prices, bids, asks):
    # Calculate the RSI divergence threshold (adjust as needed)
    rsi_divergence_threshold = 5

    # Calculate ATR using high_prices and low_prices
    atr = calculate_atr(high_prices, low_prices, close_prices)

    if imbalance_percentage >= 20:  # Positive imbalance condition
        # Check for bullish RSI divergence (oversold RSI)
        if rsi < 27:
            proposed_entry_price = bids[0][0]

            # Calculate take profit and stop loss prices
            take_profit_price, stop_loss_price = calculate_take_profit_and_stop_loss(
                proposed_entry_price,
                leverage,
                TAKE_PROFIT_PERCENTAGE,
                STOP_LOSS_PERCENTAGE
            )

            return "Validated Bullish Divergence (Long)", proposed_entry_price, take_profit_price, stop_loss_price
        else:
            return "No Entry", None, None, None
    elif imbalance_percentage <= -20:  # Negative imbalance condition
        # Check for bearish RSI divergence (overbought RSI)
        if rsi > 73:
            proposed_entry_price = asks[0][0]

            # Calculate take profit and stop loss prices
            take_profit_price, stop_loss_price = calculate_take_profit_and_stop_loss(
                proposed_entry_price,
                leverage,
                TAKE_PROFIT_PERCENTAGE,
                STOP_LOSS_PERCENTAGE
            )

            return "Validated Bearish Divergence (Short)", proposed_entry_price, take_profit_price, stop_loss_price
        else:
            return "No Entry", None, None, None
    else:
        # Check for hidden bullish divergence (higher low in RSI)
        if rsi < 30 and rsi + rsi_divergence_threshold < calculate_rsi(close_prices[-2:], atr):
            proposed_entry_price = bids[0][0]

            return "Hidden Bullish Divergence (Long)", proposed_entry_price, None, None

        # Check for hidden bearish divergence (lower high in RSI)
        if rsi > 70 and rsi - rsi_divergence_threshold > calculate_rsi(close_prices[-2:], atr):
            proposed_entry_price = asks[0][0]

            return "Hidden Bearish Divergence (Short)", proposed_entry_price, None, None
        else:
            return "No Entry", None, None, None

def create_order_with_percentage_levels(exchange, symbol, side, entry_price, leverage, amount, take_profit_percentage, stop_loss_percentage):
    try:
        # Calculate take-profit and stop-loss prices
        take_profit_price, stop_loss_price = calculate_take_profit_and_stop_loss(
            entry_price,
            leverage,
            TAKE_PROFIT_PERCENTAGE,
            STOP_LOSS_PERCENTAGE
        )

        # Create the main limit order with leverage
        main_order = exchange.create_order(
            symbol,
            type='limit',
            side=side,
            amount=amount,
            price=entry_price,
            params={
                'postOnly': True,
                'timeInForce': 'GTC',
                'leverage': leverage
            }
        )
        print("Main Order Created:", main_order)

        # Create the stop-loss order
        stop_loss_order = exchange.create_order(
            symbol,
            type='limit',
            side='sell' if side == 'buy' else 'buy',
            amount=amount,
            price=stop_loss_price
        )
        print("Stop-Loss Order Created:", stop_loss_order)

        # Create the take-profit order
        take_profit_order = exchange.create_order(
            symbol,
            type='limit',
            side='sell' if side == 'buy' else 'buy',
            amount=amount,
            price=take_profit_price
        )
        print("Take-Profit Order Created:", take_profit_order)

        return main_order, stop_loss_order, take_profit_order
    
    except Exception as e:
        print(f"Error creating orders with percentage-based levels: {e}")
        return None, None, None
    
def load_trading_signals_from_csv(file_path):
    historical_signals = []
    try:
        with open(file_path, "r", newline='') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for row in csv_reader:
                timestamp = datetime.strptime(row["Timestamp"], "%Y-%m-%d %H:%M:%S.%f")
                trading_signal = row["Trading Signal"]
                proposed_entry_price = float(row["Proposed Entry Price"]) if row["Proposed Entry Price"] else None
                order_book_imbalance = float(row["Order Book Imbalance"]) if row["Order Book Imbalance"] else None
                rsi = float(row["RSI"]) if row["RSI"] else None
                
                signal = {
                    "timestamp": timestamp,
                    "trading_signal": trading_signal,
                    "proposed_entry_price": proposed_entry_price,
                    "order_book_imbalance": order_book_imbalance,
                    "rsi": rsi
                }
                historical_signals.append(signal)
                
    except FileNotFoundError:
        pass  # The file may not exist initially, which is fine
    return historical_signals

    
# Load historical trading signals from a CSV file
historical_trading_signals = load_trading_signals_from_csv("btc_signal.csv")

# Initialize the list of trading signals with historical data
trading_signals = historical_trading_signals if historical_trading_signals else [] 

def execute_order_book_analysis(symbol, leverage, amount, take_profit_percentage, stop_loss_percentage):
    trading_signals = []  # Initialize an empty list to store trading signals

    while True:
        # Fetch OHLCV data and analyze order book
        rsi, imbalance_percentage, close_prices, high_prices, low_prices, bids, asks = fetch_ohlcv_and_analyze_order_book(symbol)
        
        # Calculate order book liquidity using dynamic range based on market volatility
        market_price, dynamic_range_percentage, lower_bound, upper_bound, total_bid_volume, total_ask_volume = calculate_order_book_liquidity_dynamic(symbol, volatility_multiplier)
       
        # Generate trading signal and proposed entry price based on RSI and order book imbalance
        trading_signal, proposed_entry_price, take_profit_price, stop_loss_price = generate_trading_signal(
            rsi,
            imbalance_percentage,
            close_prices,
            high_prices,
            low_prices,
            bids,
            asks
        )
       
        # Get the current timestamp
        timestamp = int(time.time() * 1000)
   
        # Convert timestamp to datetime
        timestamp_datetime = datetime.fromtimestamp(timestamp / 1000.0)
 
        # Create a new signal dictionary
        new_signal = {
            "timestamp": timestamp_datetime,
            "trading_signal": trading_signal,
            "proposed_entry_price": proposed_entry_price,
            "order_book_imbalance": imbalance_percentage,  # Include order book imbalance
            "rsi": rsi,  # Include RSI
            "market_price": market_price,  # Include market_price
            "dynamic_range_percentage": dynamic_range_percentage,  # Include dynamic_range_percentage
            "lower_bound": lower_bound,  # Include lower_bound
            "upper_bound": upper_bound,  # Include upper_bound
            "total_bid_volume": total_bid_volume,  # Include total_bid_volume
            "total_ask_volume": total_ask_volume,  # Include total_ask_volume
        }
        
        # Append the new signal to trading_signals
        trading_signals.append(new_signal)
 
        # Call the saving function to update the CSV file
        save_trading_signals_to_csv()
       
        # Print the relevant information
        print("Timestamp:", timestamp_datetime)
        print("Trading Signal:", trading_signal)
        if proposed_entry_price:
            print("Proposed Entry Price:", "{:.2f}".format(proposed_entry_price))
        
        # Ensure that market_price is available before accessing it
        if market_price is not None:
            print("Market Price:", "{:.2f}".format(market_price))
            print("Dynamic Range Percentage:", "{:.8f}".format(dynamic_range_percentage))
            print("Lower Bound:", "{:.8f}".format(lower_bound))
            print("Upper Bound:", "{:.8f}".format(upper_bound))
            print("Total Bid Volume within Dynamic Range:", total_bid_volume)
            print("Total Ask Volume within Dynamic Range:", total_ask_volume)

        print("=" * 50)
        
        if trading_signal != "No Entry" and proposed_entry_price:
            # Create a limit order based on the signal
            if trading_signal.startswith("Validated Bullish"):
                # Pass the desired take profit and stop loss percentages to the order creation function
                create_order_with_percentage_levels(exchange, symbol, 'buy', proposed_entry_price, leverage, amount, take_profit_percentage, stop_loss_percentage)
            elif trading_signal.startswith("Validated Bearish"):
                # Pass the desired take profit and stop loss percentages to the order creation function
                create_order_with_percentage_levels(exchange, symbol, 'sell', proposed_entry_price, leverage, amount, take_profit_percentage, stop_loss_percentage)
       
        time.sleep(60)  # Sleep for 60 seconds before fetching again


def save_trading_signals_to_csv():
    file_exists = os.path.isfile("btc_signal.csv")

    with open("btc_signal.csv", "a", newline='') as csv_file:
        fieldnames = ["Timestamp", "Trading Signal", "Proposed Entry Price", "Order Book Imbalance", "RSI", "Market Price", "Total Bid Volume", "Total Ask Volume"]
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        if not file_exists:
            csv_writer.writeheader()

        for signal in trading_signals:
            # Create a new dictionary excluding the unwanted fields
            filtered_signal = {
                "Timestamp": signal["timestamp"],
                "Trading Signal": signal["trading_signal"],
                "Proposed Entry Price": signal["proposed_entry_price"],
                "Order Book Imbalance": signal["order_book_imbalance"],
                "RSI": signal["rsi"],
                "Market Price": signal.get("market_price", None),  # Check for None to handle old data
                "Total Bid Volume": signal.get("total_bid_volume", None),  # Check for None to handle old data
                "Total Ask Volume": signal.get("total_ask_volume", None)  # Check for None to handle old data
            }
            csv_writer.writerow(filtered_signal)

 
# Call execute_order_book_analysis directly for your single symbol analysis
execute_order_book_analysis(symbol_to_analyze, leverage, amount, TAKE_PROFIT_PERCENTAGE, STOP_LOSS_PERCENTAGE)

# After the while loop ends (script termination), save the updated trading signals to the CSV file
save_trading_signals_to_csv()
